In [1]:
import tensorflow as tf 
import pathlib
import numpy as np
from matplotlib import pyplot
from matplotlib import image
import glob

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def download_oasis ():
    
    #download oasis brain MRI data
    dataset_url = "https://cloudstor.aarnet.edu.au/plus/s/n5aZ4XX1WBKp6HZ/download"
    data_dir = tf.keras.utils.get_file(origin=dataset_url,fname='oa-sis' ,untar=True)
    data_dir = pathlib.Path(data_dir)
    
    # unzip data to current directory 
    print (data_dir)
    ! unzip /root/.keras/datasets/oa-sis.tar.gz

In [ ]:
def load_training (path):
    # load training images (non segmented) in the path and store in numpy array
    image_list = []
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        image_list.append(im)

    print('train_X shape:',np.array(image_list).shape)
    train_set = np.array(image_list, dtype=np.float32)
    return train_set


In [ ]:
def process_training (data_set):
    # the method normalizes training images and adds 4th dimention 

    train_set = data_set
    train_set = (train_set - np.mean(train_set))/ np.std(train_set)
    train_set= (train_set- np.amin(train_set))/ np.amax(train_set- np.amin(train_set))
    train_set = train_set [:,:,:,np.newaxis]
    
    return train_set

In [ ]:
def load_labels (path):
    # loads labels images and map pixel values to class indices and convert image data type to unit8 

    n_classes = 4
    image_list =[]
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        one_hot = np.zeros((im.shape[0], im.shape[1]))
        for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :][im == unique_value] = i
        image_list.append(one_hot)

    print('train_y shape:',np.array(image_list).shape)
    labels = np.array(image_list, dtype=np.uint8)
    
    pyplot.imshow(labels[2])
    pyplot.show()

    return labels

In [ ]:
def process_labels(seg_data):
    # one hot encode label data and convert to numpy array
    onehot_seg_data = []
    for n in range(seg_data.shape[0]): 
      im = seg_data[n]
      n_classes = 4
      one_hot = np.zeros((im.shape[0], im.shape[1], n_classes),dtype=np.uint8)
      for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :, i][im == unique_value] = 1
    onehot_seg_data.append(one_hot)
    
    onehot_seg_data =np.array(onehot_seg_data)
    print (onehot_seg_data.dtype)
    #print (np.unique(onehot_validate_Y))
    print (onehot_seg_data.shape)

    return onehot_seg_data


In [1]:
# U-NET final model w on2Dtranspose and batch normalization after activation
# questions do I need to use shuffle =true in fit module ?
import tensorflow as tf

def unet_model ():
    filter_size=16 
    input_layer = tf.keras.Input((256,256,1))
    
    pre_conv = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same")(input_layer)
    pre_conv = tf.keras.layers.LeakyReLU(alpha=.01)(pre_conv)


# context module 1 pre-activation residual block
    conv1 = tf.keras.layers.BatchNormalization()(pre_conv)
    conv1 = tf.keras.layers.LeakyReLU(alpha=.01)(conv1)
    conv1 = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same" )(conv1) 
    conv1 = tf.keras.layers.Dropout(.3) (conv1)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.LeakyReLU(alpha=.01)(conv1)
    conv1 = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same")(conv1)    
    conv1 = tf.keras.layers.Add()([pre_conv,conv1])
    
# downsample and double number of feature maps   
    pool1 = tf.keras.layers.Conv2D(filter_size * 2, (3,3), (2,2) , padding='same')(conv1)
    pool1 = tf.keras.layers.LeakyReLU(alpha=.01)(pool1)
    
# context module 2
    conv2 = tf.keras.layers.BatchNormalization()(pool1)
    conv2 = tf.keras.layers.LeakyReLU(alpha=.01)(conv2)
    conv2 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(conv2)
    conv2 = tf.keras.layers.Dropout(.3) (conv2)  
    conv2 = tf.keras.layers.BatchNormalization()(conv2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=.01)(conv2)
    conv2 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(conv2)
    conv2 = tf.keras.layers.Add()([pool1,conv2])

# downsample and double number of feature maps
    pool2 = tf.keras.layers.Conv2D(filter_size*4, (3,3),(2,2), padding='same')(conv2)
    pool2 = tf.keras.layers.LeakyReLU(alpha=.01)(pool2)

# context module 3
    conv3 = tf.keras.layers.BatchNormalization()(pool2)
    conv3 = tf.keras.layers.LeakyReLU(alpha=.01)(conv3)
    conv3 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(conv3)
    conv3 = tf.keras.layers.Dropout(.3) (conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=.01)(conv3)
    conv3 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(conv3)
    conv3 = tf.keras.layers.Add()([pool2,conv3])

# downsample and double number of feature maps
    pool3 = tf.keras.layers.Conv2D(filter_size*8, (3,3),(2,2),padding='same')(conv3)
    pool3 = tf.keras.layers.LeakyReLU(alpha=.01)(pool3)

# context module 4
    conv4 = tf.keras.layers.BatchNormalization()(pool3)
    conv4 = tf.keras.layers.LeakyReLU(alpha=.01)(conv4)
    conv4 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(conv4)
    conv4 = tf.keras.layers.Dropout(.3) (conv4)
    conv4 = tf.keras.layers.BatchNormalization()(conv4)
    conv4 = tf.keras.layers.LeakyReLU(alpha=.01)(conv4)
    conv4 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(conv4)
    conv4 = tf.keras.layers.Add()([pool3,conv4])
    print ("conv4",conv4.shape)

# downsample and double number of feature maps
    pool4 = tf.keras.layers.Conv2D(filter_size*16, (3,3),(2,2),padding='same')(conv4)
    pool4 = tf.keras.layers.LeakyReLU(alpha=.01)(pool4) 

# context module 5
    # Middle
    convm = tf.keras.layers.BatchNormalization()(pool4)
    convm = tf.keras.layers.LeakyReLU(alpha=.01)(convm)
    convm = tf.keras.layers.Conv2D(filter_size * 16, (3, 3), padding="same")(convm)
    convm = tf.keras.layers.Dropout(.3) (convm)
    convm = tf.keras.layers.BatchNormalization()(convm)
    convm = tf.keras.layers.LeakyReLU(alpha=.01)(convm)
    convm = tf.keras.layers.Conv2D(filter_size * 16, (3, 3), padding="same")(convm)
    convm = tf.keras.layers.Add()([pool4,convm])
    print ("convm",convm.shape)

#upsampling module 1
    deconv4 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(convm)
    deconv4 = tf.keras.layers.Conv2D (filter_size *8, (3, 3) , padding="same")(deconv4)
    deconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv4) 
    print ("upsample 1",deconv4.shape)

#concatatinate layers 
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4], axis=3)
    print ("concat 1",uconv4.shape)

#localization module 1
    uconv4 = tf.keras.layers.Conv2D(filter_size * 16, (3, 3) , padding="same")(uconv4)
    uconv4 = tf.keras.layers.BatchNormalization()(uconv4)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv4)
    uconv4 = tf.keras.layers.Conv2D(filter_size * 8, (1, 1), padding="same")(uconv4)
    uconv4 = tf.keras.layers.BatchNormalization()(uconv4)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv4)
    print ("localization 1",uconv4.shape)

#upsampling module 2
    deconv3 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(uconv4)
    deconv3 = tf.keras.layers.Conv2D (filter_size *4, (3, 3) , padding="same")(deconv3)
    deconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv3) 
    #print ("upsample 2",deconv3.shape)
  

# concatatinate layers  
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3], axis=3)
    #print ("concat 3",uconv3.shape)

# localization module 2
    uconv3 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(uconv3)
    uconv3 = tf.keras.layers.BatchNormalization()(uconv3)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv3)
    uconv3 = tf.keras.layers.Conv2D(filter_size * 4, (1, 1), padding="same")(uconv3)
    uconv3 = tf.keras.layers.BatchNormalization()(uconv3)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv3)
    #print ("localization 2",uconv3.shape)

    seg3 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same' )(uconv3)
    seg3 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(seg3)
    #print ("seg3",seg3.shape)

# Upsample module 3
    deconv2 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(uconv3)
    deconv2 = tf.keras.layers.Conv2D (filter_size *2, (3, 3) , padding="same")(deconv2)
    deconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv2)
    #print ("upsample 3",deconv2.shape)

# concatination layer 
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2], axis=3)
    #print ("concat 3",uconv2.shape)

# localization module 3
    uconv2 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(uconv2)
    uconv2 = tf.keras.layers.BatchNormalization()(uconv2)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(filter_size * 2, (1, 1), padding="same")(uconv2)
    uconv2 = tf.keras.layers.BatchNormalization()(uconv2)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv2)
    #print ("localization 3",uconv2.shape)

    seg2 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same')(uconv2)
    #print ("seg2",seg2.shape)
    seg_32 = tf.keras.layers.Add()([seg3,seg2])

    seg_32 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(seg_32)
    #print ("seg_32",seg_32.shape)

# Upsample module 4
    deconv1 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='nearest')(uconv2)
    deconv1 = tf.keras.layers.Conv2D (filter_size *1, (3, 3) , padding="same")(deconv1)
    deconv1 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv1)
    #print ("upsample 4",deconv1.shape)

# concatination layer
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1], axis=3)
    #print ("concat 4",uconv1.shape)

    uconv1 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(uconv1)
    #uconv1 = tf.keras.layers.BatchNormalization()(uconv1)
    uconv1 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv1)
    
    #print ("conv before last activation",uconv1.shape)
    
    seg1 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same' )(uconv1)
    #print ("seg 1",seg1.shape)

    seg_sum = tf.keras.layers.Add()([seg1,seg_32])


    output_layer = tf.keras.layers.Conv2D(4, (3,3), padding='same' ,activation="softmax")(seg_sum)
    
    model = tf.keras.Model( input_layer , outputs=output_layer)


    
    return model

#'CategoricalCrossentropy'


In [2]:
# test script load and preprocess training images

# download oasis data and unzip files 
download_oasis()

# load training data set  
train_X = load_training ('/content/keras_png_slices_data/keras_png_slices_train')

# check loaded image
pyplot.imshow(train_X[2])
pyplot.show()

# pre- process training dataset 
train_X = process_training(train_X)

# load validaton data set and process it 
validate_X = load_training ('/content/keras_png_slices_data/keras_png_slices_validate')

# check loaded images
pyplot.imshow(validate_X[2])
pyplot.show()

# pre process validation data set
validate_X = process_training(validate_X)


# load test data set and process it 
test_X = load_training ('/content/keras_png_slices_data/keras_png_slices_test')

# check loaded images
pyplot.imshow(test_X[2])
pyplot.show()

# pre process test data set
test_X = process_training(test_X)


conv4 (None, 32, 32, 128)
convm (None, 16, 16, 256)
upsample 1 (None, 32, 32, 128)
concat 1 (None, 32, 32, 256)
localization 1 (None, 32, 32, 128)


In [ ]:

# load and pre process segmented training set 
train_Y = load_labels('/content/keras_png_slices_data/keras_png_slices_seg_train')
train_Y = process_labels(train_Y)

# check loaded images
pyplot.imshow(train_Y[2,:,:,3])
pyplot.show()


# load and pre process segmented validation set 
validate_Y = load_labels('/content/keras_png_slices_data/keras_png_slices_seg_validate')
validate_Y = process_labels(validate_Y)
 
# check loaded images
pyplot.imshow(validate_Y[2,:,:,3])
pyplot.show()

# load and pre process segmented test set 
test_Y = load_labels('/content/keras_png_slices_data/keras_png_slices_seg_test')
test_Y = process_labels(test_Y)
 
# check loaded images
pyplot.imshow(test_Y[2,:,:,3])
pyplot.show()

In [ ]:
# create model and set training paramters 

# create a model instance and set training paramters 

model = unet_model()
opt= tf.keras.optimizers.Adam (learning_rate=.0005)
model.compile (optimizer=opt, loss= 'CategoricalCrossentropy' , metrics=['accuracy'])

# set early stop criteria 
ES = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy',min_delta=.0001, patience=40, verbose=0, mode='max', restore_best_weights=True)

# record history of training to display loss over ephocs 
history = model.fit(train_X, train_Y,  validation_data= (validate_X, validate_Y) ,batch_size=32,shuffle='True',epochs=2, callbacks=[ES])

# evaluate against testing data 
model.evaluate(test_X,test_Y)

# save trained model weights 
model.save_weights('/content/drive/My Drive/modelweights/unet8')


# plot training and validation loss 
pyplot.title('Categorical Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='test')
pyplot.legend(('training','validation'))


In [ ]:
loaded_model = unet_model()
opt= tf.keras.optimizers.Adam (learning_rate=.0005)
loaded_model.compile (optimizer=opt, loss= 'CategoricalCrossentropy' , metrics=['accuracy'])
loaded_model.load_weights('/content/drive/My Drive/modelweights/unet8')
loaded_model.evaluate (test_X,test_Y)